In [1]:
from dotenv import load_dotenv
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage, ToolMessage, SystemMessage



MODEL = "mistral-nemo"
TEMPERATURE = 0
load_dotenv('../.env', override=True)

from research_agent import create_agent
agent = create_agent(
    model=MODEL,
    builtin_tools=['wikipedia', 'web_search', 'calculator']
)

In [2]:
claim = "The creator of Python is Guido van Rossum."
results = agent.invoke({'claim': claim})
for result in results['messages']:
    result.pretty_print()

Using first search result Guido van Rossum!


TypeError: 'AIMessage' object is not subscriptable

In [ ]:
tool_message = results['messages'][-2]
print(type(tool_message))
print(dir(tool_message))
print(type(tool_message.content))

import json
obj = json.loads(tool_message.content)
print(type(obj))


In [ ]:
tool_message

In [ ]:
claim = "2 raised to the 10th power is 1024."
results = agent.invoke({'claim': claim})

In [ ]:
for message in results['messages']:
    message.pretty_print()

The tool calls are in AI Messages. That is, an `AIMessage` will have a `.tool_calls` attribute which is a list of tool calls or `None`

In [ ]:
from pprint import pprint
print(type(results['messages'][2]))
pprint(results['messages'][2].tool_calls)

In [ ]:
print(results['evidence'])

In [ ]:
# from langchain_core.messages import HumanMessage, SystemMessage
claim = "1/3 is bigger than 1/4."
# initial_state = {"claim": claim}
# final_state = graph.invoke(initial_state)

messages = [sys_msg]
messages = react_graph.invoke({"messages": messages, "claim": claim})

In [ ]:
for m in messages['messages']:
    m.pretty_print()

In [ ]:
from pprint import pprint
pprint(messages)

In [ ]:
response = react_graph.nodes['assistant'].invoke(
    {"messages": [HumanMessage(content="hi there")]}
)

In [ ]:
print(response)